In [1]:
import numpy as np
import os
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, f_classif, mutual_info_classif
import time
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, LassoCV, RandomizedLasso
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold #, GridSearchCV, cross_val_score

%matplotlib inline 
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib
from mpl_toolkits.mplot3d import Axes3D #, axes3d
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import operator
# import winsound


datapath = 'tmp/'
featurepath = 'features/1024_1/'
XYfile = datapath + featurepath + 'newfeatures_1024_1_10_10000_XY.npz'
############# GATHERING into one complete array
#X = np.load(XYfile)['X']
with np.load(XYfile) as inpf:
    X = inpf['X']
    Y = inpf['Y']
#Y = np.load(XYfile)['Y']

   
# print ('gathered features: ', X[0].shape, Y[0].shape, X[1].shape, Y[1].shape, X[2].shape, Y[2].shape, np.sum(Y[2]))

In [2]:
# copied 
window = 1024
standardScaler = preprocessing.StandardScaler()
# x = np.copy(X[0])
X[0] = standardScaler.fit_transform(X[0])
## Time Domain Phinyomark feats
featnames = ['intsgnl', 'meanabs', 'meanabsslp', 'ssi', 'var', 'rms', 'rng', 'wavl', 'zerox', 'ssc', 'wamp', 
             'shist1', 'shist2', 'shist3']                                                   # 11+3{shist}
## Frequency Domain Phinyomark feats
featnames += ['arco1', 'arco2', 'arco3', 'mnf', 'mdf', 'mmnf', 'mmdf']                       # 3{arco}+4{mf}
featnames += ['reFFT{:03d}'.format(i) for i in range(window//2+1)]                            # samples/2+1{RF}
featnames += ['imFFT{:03d}'.format(i) for i in range(window//2+1)]                            # samples/2+1{IF}
## Time Domain Golz feats
featnames += ['meanv', 'stdr', 'mx', 'rngx', 'rngy', 'med', 'hjorth', 'sentr', 'se', 'ssk']  # 10
featnames += ['acrol{:04d}'.format(i) for i in range(window)]                                # samples{acrol}
## Frequency Domain Golz feats
featnames += ['amFFT{:03d}'.format(i) for i in range(window//2+1)]                            # samples/2+1{AF}
featnames += ['phFFT{:03d}'.format(i) for i in range(window//2+1)]                            # samples/2+1{PF}
featnames += ['ffaf']                                                                        # 1{ffaf}

In [7]:
######################################## FEATURE SELECTION #########################################################
# feature names


# import joblib
# from joblib import Parallel, delayed, Memory
shift =1 ; keepfromshift = 10; samplesperdataset = 10000

tmpind = {}
tmpind[2] = range(X[0].shape[1])
tmpind[0] = range(X[0].shape[1]//2)
tmpind[1] = range(X[0].shape[1]//2,X[0].shape[1])
tmpind = np.array([i for _,i in tmpind.items()])

featpath = datapath+'features/'+str(window)+'_'+str(shift)+'/'
featname = 'newfeatures'+'_'+str(window)+'_'+str(shift)+'_'+str(keepfromshift)+'_'+str(samplesperdataset)



numfeat = 10 # number of features to show
nfeat = 1000 # number of features to keep
#namesid = ['sf{:04d}'.format(i) if i<X[0].shape[1]/2 else 'ftn{:04d}'.format(i) for i in range(X[0].shape[1])]
namesid = [['sf{:04d}'.format(i) for i in range(X[0].shape[1]//2)],
           ['ftn{:04d}'.format(i) for i in range(X[0].shape[1]//2)],
           ['sf{:04d}'.format(i) if i<X[0].shape[1]//2 else 'ftn{:04d}'.format(i%X[0].shape[1]//2) for i in range(X[0].shape[1])]]
namesf = [['sf_{}'.format(featnames[i]) for i in range(X[0].shape[1]//2)],
         ['ftn_{}'.format(featnames[i]) for i in range(X[0].shape[1]//2)],
         ['sf_{}'.format(featnames[i]) if i<X[0].shape[1]//2 else 'ftn_{}'.format(featnames[i%X[0].shape[1]//2]) for i in range(X[0].shape[1])]]
########## use RandomizedLasso of MutualInfo as the model to select features and find their importances ############
featselfile = featpath+featname+'_featselall'+'.npz'
tmpskip = [int(len(tmpind[i])/3000) for i in range(len(tmpind))]
#tmpskip = [1 for i in range(3)]
start_time = time.time()
if os.path.isfile(featselfile):
#     print("hey yooooo")
    #rlasso = np.load(featselfile)['rlasso'].tolist()
    featsel = np.load(featselfile)['featsel'].tolist()
#     print("Selected Features FOUND PRECOMPUTED! Loading DONE in: %s seconds " % (time.time() - start_time))
else:                                                                                                    
    with warnings.catch_warnings():
#         print("You're in for a long ride")
        warnings.simplefilter('ignore', UserWarning)
        warnings.simplefilter('ignore', ConvergenceWarning)
        featsel = [[SelectKBest(mutual_info_classif,'all').fit(X[i][:-1:tmpskip[i],tmpind[j]],Y[i][:-1:tmpskip[i]]) 
                    for j in range(3)] for i in range(1)]
#         print("Selected Features NOT FOUND PRECOMPUTED! Selection DONE in: %s seconds " % (time.time() - start_time))
        # loop for all datasets (12,3,all) and all features (sf,ftn,all)
        #featsel = [[RandomizedLasso(fit_intercept=False,normalize=False).fit(X[i][:-1:tmpskip[i],tmpind[j]],Y[i][:-1:tmpskip[i]]) 
        #Parallel(n_jobs=-1)([delayed(feat) (p[k:k+window],*featparam) for k in range(0,len(p)-window,shift)])
#         featsel = [Parallel(n_jobs=-1)([delayed(SelectKBest(mutual_info_classif,'all').fit) 
#                     (X[i][:-1:tmpskip[i],tmpind[j]],Y[i][:-1:tmpskip[i]])
#                     for i,j in itertools.product(range(len(X)),range(3))])]
#         featsel = np.array([[featsel[0][i*len(range(3))+j] for j in range(3)] for i in range(len(X))])
#                    for j in range(3) for i in range(len(X))]
        
        #np.savez(featselfile,rlasso=np.array(rlasso))
        np.savez(featselfile,featsel=featsel)
# rank features
# print "----> Features sorted by their rank (std norm):"
bestix = {}
wrstix = {}
# for i in range(len(X)): # for all diff data sets
# Im only using the first one so the for above doesnt apply
for i in range (3):
    bestix[i] = {}
    wrstix[i] = {}
    for j in range(3): # for all diff feature sets
#         tmp = sorted(zip(map(lambda x: round(x,4), featsel[i][j].scores_),namesf[j]),reverse=True)
#         print(j)
        bestix[i][j] = np.array(featsel[i][j].scores_).argsort()[:][::-1]
        #wrstix[i][j] = np.array(featsel[i][j].scores_).argsort()[:nfeat][::1]
        wrstix[i][j] = np.array(featsel[i][j].scores_).argsort()[:][::1]
#         print 'Best '+str(numfeat)+': ',tmp[:numfeat], bestix[i][j].shape, bestix[i][j][:numfeat]
#         print 'Worst '+str(numfeat)+': ',tmp[-numfeat:], wrstix[i][j].shape, wrstix[i][j][:numfeat]
bestix = np.array([[ij for _,ij in i.items()] for _,i in bestix.items()])
wrstix = np.array([[ij for _,ij in i.items()] for _,i in wrstix.items()])

In [8]:
np.array(featsel).shape

(3, 3)

In [10]:

############################### DIMENSIONALITY REDUCTION VIA FEATURE SELECTION #####################################
# fig = plt.figure(figsize=(8,6))
r,c = np.array(featsel).shape[0],np.array(featsel).shape[1]
Xfs = {}
# matplotlib.rcParams['text.usetex'] = True
for i in range(r):
    Xfs[i] = {}
    for j in range(c):
        Xfs[i][j] = np.array(featsel[i][j].transform(X[i][:,tmpind[j]]))[:,bestix[i][j][:nfeat]]
#         print (Xfs[i][j].shape)
#         ax = fig.add_subplot(r,c,i*c+j+1)
#         ax.plot(featsel[i][j].scores_,'b')
#         ax.scatter(bestix[i][j][:nfeat],featsel[i][j].scores_[bestix[i][j][:nfeat]],color='r')
#         if i==0 and j==0:
#             ax.set_title(r'$\zeta_1=\sigma(|f|)$')
#             ax.set_ylabel(r'$d_1$')
#         if i==0 and j==1:
#             ax.set_title(r'$\zeta_2=\sigma(|f_t/f_n|)$')
#         if i==0 and j==2:
#             ax.set_title(r'$\zeta_3=\zeta_1\bigcup \zeta_2$')
#         if i==1 and j==0:
#             ax.set_ylabel(r'$d_2$')
#         if i==2 and j==0:
#             ax.set_ylabel(r'$d_3=d_1\bigcup d_2$')
#         if i==2:
#             ax.set_xlabel('features')
Xfs = np.array([[i for _,i in Xfsi.items()] for _,Xfsi in Xfs.items()])
#plt.show()
# savefig('plots/'+featname+'_featsel.png', bbox_inches='tight')
# savefig('plots/featureselection.eps',format='eps',dpi=50,bbox_inches='tight')

In [11]:
################################### DIMENSIONALITY REDUCTION #######################################################
featdimredfile = featpath+featname+'_dimred'+'.npz'
ncomp = 50
tmpskip = 1
# print(Xfs[0].shape[0], Xfs[0].shape[1])

if os.path.isfile(featdimredfile):
    inpdimred = np.load(featdimredfile)
    pca = inpdimred['pca'].tolist()
    Xpca = inpdimred['Xpca']
    tmpix = inpdimred['tmpix']
else:
    # PCA
    r,c = Xfs.shape[0],Xfs.shape[1]
    Xpca,pca,tmpix = {},{},{}
    for i in range(r):
        Xpca[i],pca[i] = {},{}
        tmpix[i] = range(0,Xfs[i][0].shape[0],tmpskip)
        for j in range(c):
            pca[i][j] = PCA(n_components=ncomp, random_state=0)
            pca[i][j].fit(Xfs[i][j][tmpix[i],:])
            Xpca[i][j] = pca[i][j].transform(Xfs[i][j][tmpix[i],:])
    pca = np.array([[i for _,i in pcai.items()] for _,pcai in pca.items()])
    Xpca = np.array([[i for _,i in Xpcai.items()] for _,Xpcai in Xpca.items()])
    tmpix = np.array([i for _,i  in tmpix.items()])
    np.savez(featdimredfile,pca=pca,Xpca=Xpca,tmpix=tmpix)
# for i in range(r):
#     for j in range(c):
#         print (Xpca[i][j].shape, np.array(tmpix[i]).shape ,(pca[i][j].explained_variance_ratio_))
#         ################################## VISUALIZATION OF DIMENSIONALITY REDUCTION ###############################
#         fig = plt.figure(figsize=(6,2))
#         fig.suptitle(str(i)+str(j), fontsize=14, fontweight='bold')
#         #ax = Axes3D(fig)
#         #fig = plt.figure()
#         #ax = fig.add_subplot(111, projection='3d')
#         ax = fig.add_subplot(141, projection='3d')
#         #ax = fig.add_subplot(141)
#         ax.scatter(Xpca[i][j][:,0],Xpca[i][j][:,1],Xpca[i][j][:,2],c=Y[i][:,np.newaxis]*np.array([[1,1,0]]))
#         ax = fig.add_subplot(142)
#         ax.scatter(Xpca[i][j][:,0],Xpca[i][j][:,1],c=Y[i][:,np.newaxis]*np.array([[1,1,0]]))
#         ax = fig.add_subplot(143)
#         ax.scatter(Xpca[i][j][:,0],Xpca[i][j][:,2],c=Y[i][:,np.newaxis]*np.array([[1,1,0]]))
#         ax = fig.add_subplot(144)
#         ax.scatter(Xpca[i][j][:,1],Xpca[i][j][:,2],c=Y[i][:,np.newaxis]*np.array([[1,1,0]]))

In [12]:
######################################## DATASETS PREPARATION ######################################################
datasets = [(Xpca[i][j],Y[i][tmpix[i]]) for j in range(c) for i in range(r)]
# print (np.array(datasets).shape)
for ds in datasets:
    X1,Y1 = ds
#     print (X1.shape,Y1.shape)
#     tems = np.sum(Y1)
#     print(tems)
#     plt.hist(Y1)


In [13]:
X1.shape, Y1.shape, sum(Y1)/len(Y1)

((20149, 50), (20149,), 0.47114000694823566)

In [15]:
%%time
# Nested CV. 
# The basic idea is that cross-validation is used to assess the performance of a method for fitting a model, 
# not of the model itself. If you need to perform model selection, then you need to perform that independently 
# in each fold of the cross-validation procedure, as it is an integral part of the model fitting procedure. 
# If you use a cross-validation based model selection procedure, this means you end up with nested cross-validation.
# It is helpful to consider the purpose of each cross-validation - one is for model selection, the other for performance
# estimation. I would make my final model by fitting the model (including model selection) to the whole dataset, after
# using nested cross-validation to get an idea of the performance I could reasonably expect to get from that model.



classifiers = [#SVC(kernel = "linear")]#,
       SVC(gamma = 2.5, C = 10),#, 
       GaussianNB(),
       KNeighborsClassifier(4),
       KNeighborsClassifier(5),
       MLPClassifier(alpha = 1)]

X,y = datasets[1]

# classifiers = [SVC( gamma = 2.5)]
# print(len(classifiers))
n_outer_folds = 5
n_inner_folds = 5
# outer_scores = np.zeros((len(top_classifiers), n_outer_folds)) #use this if you wanna keep the score of multiple "best" classifiers from the inner cv
outer_scores  = np.zeros(n_outer_folds)
# print(outer_scores.shape, inner_scores.shape, inner_mean_scores.shape)
state = 42 
x_tot = np.copy(X) ; y_tot = np.copy(y)


# outer_scores = []
#outer loop i.e. the one that evaluates the inner "best" model
outer_kfold = KFold(n_splits = n_outer_folds, shuffle = True, random_state = state)
outer = outer_kfold.split(x_tot)
# print("chkpoint 2")

for fold_out, (train_ind_out, test_ind_out) in enumerate(outer):
    print("Outer loop %d" %fold_out)
    x_trn_out, x_tst_out = x_tot[train_ind_out], x_tot[test_ind_out]
    y_trn_out, y_tst_out = y_tot[train_ind_out], y_tot[test_ind_out]  
#     print("outer data split")
    inner_scores = np.zeros((len(classifiers),n_inner_folds))
    inner_mean_scores = np.zeros(len(classifiers))
    for cl_ind,clf in enumerate(classifiers):
        print("Classifier %d" %cl_ind)
        inner_kfold = KFold( n_splits = n_inner_folds, shuffle = True, random_state = state)
        inner = inner_kfold.split(x_trn_out)
#         print("classifier loaded")
        
        for fold_in, (train_ind_in, test_ind_in) in enumerate(inner): 
            print("Inner fold %d " %fold_in)
            x_trn_in, x_tst_in = x_trn_out[train_ind_in], x_trn_out[test_ind_in]
            y_trn_in, y_tst_in = y_trn_out[train_ind_in], y_trn_out[test_ind_in]
#             print("inner data split")
            clf.fit(x_trn_in, y_trn_in)
#             print("clf fit")
            
            #inner scores calculated for every inner fold
            inner_scores[cl_ind][fold_in] = clf.score(x_tst_in, y_tst_in)
#             print("inner scores calculated")
#         print(inner_scores)
    # get the mean performance for every classifier
    inner_mean_scores = np.mean(inner_scores, axis = 1)
#     print("inner mean score calculated")
    print(inner_mean_scores)
        
#     get the index of the best classifier for inner cv
    best_clf = inner_mean_scores.argmax()
    best_mean_score = inner_mean_scores.max()
    print("Best classifier for outer fold %d is %s " %(fold_out, classifiers[best_clf]))
#     print("Best classifier for outer fold %d is %d " %(fold_out, best_clf))
        
    #fit the best classifier on the outer test data
    classifiers[best_clf].fit(x_trn_out, y_trn_out)
    #get the outer score 
    outer_scores[fold_out] = classifiers[best_clf].score(x_tst_out, y_tst_out)
    print("Inner score: %f VS Outer score: %f" %(best_mean_score, outer_scores[fold_out]))
        
        
        


Outer loop 0
Classifier 0
Inner fold 0 
Inner fold 1 
Inner fold 2 
Inner fold 3 
Inner fold 4 
Classifier 1
Inner fold 0 
Inner fold 1 
Inner fold 2 
Inner fold 3 
Inner fold 4 
Classifier 2
Inner fold 0 
Inner fold 1 
Inner fold 2 
Inner fold 3 
Inner fold 4 
Classifier 3
Inner fold 0 
Inner fold 1 
Inner fold 2 
Inner fold 3 
Inner fold 4 
Classifier 4
Inner fold 0 
Inner fold 1 
Inner fold 2 
Inner fold 3 
Inner fold 4 
[ 0.78668755  0.83391089  0.83453147  0.84767015  0.87580578]
Best classifier for outer fold 0 is MLPClassifier(activation='relu', alpha=1, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False) 
Inner score: 0.875806 VS Outer score: 0.8647